In [2]:
import ee
import sys
sys.path

['/usr/src/app/notebooks',
 '/usr/src/app',
 '/usr/src/app/gee_lib',
 '/usr/src/app/ex_ante_lib',
 '/usr/src/app/forestry_carbon_arr',
 '/usr/src/app/fastapi-gee-service',
 '/usr/local/lib/python310.zip',
 '/usr/local/lib/python3.10',
 '/usr/local/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/site-packages']

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from google.auth import default
creds, project = default()

In [5]:
# !gcloud projects list
# !gcloud config list
!gcloud auth list



No credentialed accounts.

To login, run:
  $ gcloud auth login `ACCOUNT`



In [6]:
# !earthengine authenticate

In [7]:
# !gcloud config set project ee-iwansetiawan

In [8]:
# !pip install earthengine-api --upgrade


In [9]:
import os
import ee
from gee_lib.osi.auth import initialize_gee, check_gee_initialization

# ee.Initialize(project=os.getenv('GOOGLE_CLOUD_PROJECT'))
# ee.Initialize(project='ee-iwansetiawan')

# Initialize GEE
# authenticate = initialize_gee(project_id=os.getenv('GOOGLE_CLOUD_PROJECT'), use_service_account=False)
authenticate = initialize_gee(project_id='ee-iwansetiawan',use_service_account=True)

# Verify it's working
if check_gee_initialization():
    print("Ready to use GEE!")

✓ GEE Initialized successfully
  Service Account: earth-engine-land-eligibility@ee-iwansetiawan.iam.gserviceaccount.com
  Project ID: ee-iwansetiawan
  Credentials Path: /usr/src/app/user_id.json


In [10]:
# !gcloud --version

# os.getenv('GOOGLE_CLOUD_PROJECT')

from google.cloud import storage
client = storage.Client()
print(client.project)


validation-469814


In [11]:
# !gcloud.storage.buckets.list

In [12]:
import json
import os

### need improvement on how to manage this config, we also in the config the path to the static file
# maybe we need to not put the path in the config.json, don't know yet
# Move to the parent directory of the current script
parent_dir = os.getcwd()

# Construct the absolute path to the JSON file in the 'input' folder
json_path= os.path.join(parent_dir, '00_input', 'balaban_conf.json')

# Read and load the JSON data from the file
with open(json_path, 'r') as file:
    config = json.load(file)

print('config---> ',config)

config--->  {'I_satellite': 'Sentinel', 'pca_scaling': 1, 'tileScale': 2, 'AOI_path': './00_input/LPHD Belaban Rayak.shp', 'OID_field_name': 'id', 'input_training': './00_input/traning_point_merged_belaban.shp', 'algo_ml_selected': 'gbm', 'date_start_end': ['2024-1-1', '2024-5-31'], 'project_name': 'belaban_rayak', 'super_pixel_size': 3, 'region': 'asia', 'pixel_number': 3, 'year_start_loss': 14, 'tree_cover_forest': 30, 'band_name_image': 'Class', 'cloud_cover_threshold': 60, 'crs_input': 'EPSG:4326', 'IsThermal': False, 'fcd_selected': 21, 'high_forest': 65, 'yrf_forest': 45, 'shrub_grass': 45, 'open_land': 30, 'ndwi_hi_sentinel': 0.05, 'ndwi_hi_landsat': 0.1, 'ndwi_hi_planet': -0.2}


In [13]:
## import modules
import osi
import pandas as pd
import geemap

from osi.utils.main import validate_aoi
# convert the modules for image collection (cloudless masking, compositing, reducer etc)
from osi.image_collection.main import ImageCollection
from osi.spectral_indices.spectral_analysis import SpectralAnalysis
from osi.spectral_indices.utils import normalization_100
from osi.hansen.historical_loss import HansenHistorical
from osi.classifying.assign_zone import AssignClassZone
from osi.legends.utils import convert_to_legend_items
from osi.legends.main import LegendsBuilder
from osi.obia.main import OBIASegmentation
from osi.ml.main import LandcoverML
from osi.fcd.main_fcd import FCDCalc
from osi.pca.pca_gee import PCA
from osi.hansen.historical_loss import HansenHistorical
from osi.classifying.assign_zone import AssignClassZone
from osi.arcpy.utils import safe_get_data_source


AOIt_shp_plot = geemap.shp_to_ee(config['AOI_path'])
crs_input = config['crs_input']
I_satellite = config['I_satellite']
project_name = config['project_name']

start_date = config['date_start_end'][0]
end_date = config['date_start_end'][1]

layer_name_image_mosaick = f'image_mosaick_result_ee_{project_name}'

AOI = AOIt_shp_plot
config['AOI'] = AOI

ndwi_hi = 0.1
if config['I_satellite'] == 'Landsat':
    ndwi_hi = config['ndwi_hi_landsat']
elif I_satellite == 'Sentinel':
    ndwi_hi = config['ndwi_hi_sentinel']
elif I_satellite == 'Planet':
    ndwi_hi = config['ndwi_hi_planet']

### Masking and overlay and area helper Make an image out of the AOI area attribute -> convert featurecollection into raster (image) for overlaying tools
OID = config['OID_field_name']
AOI_img = AOI.filter(ee.Filter.notNull([OID])).reduceToImage(
    properties= [OID],
    reducer= ee.Reducer.first()
)

In [14]:
# now starting to do analysis
# initiate instance class for the image collection and later mosaicking
classInputCollection = ImageCollection(I_satellite=I_satellite,
                                       AOI=AOI, 
                                       date_start_end=config['date_start_end'], 
                                       cloud_cover_threshold = config['cloud_cover_threshold'],
                                       region=config['region'])

In [15]:
# run the method from image collection loaded, cloudless compositing until to image_mosaick
image_mosaick = classInputCollection.image_mosaick()

selecting Sentinel images


In [16]:
from gee_lib.osi.utils import generate_map_id

# Generate Map IDs
### Defining params visualization and layer name
layer_name = f'{I_satellite} mosaicked - {start_date}-{end_date} VegColor'

if config['I_satellite'] == 'Planet':
    # true color {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}
    # nir veg color {"bands":["red","nir","blue"],"min":0,"max":0.6,"gamma":1.5 }
    vis_param = {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}
else:
    vis_param = {'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5}

layers_data = {'project_name': 'Sentinel-2 Cloudless Composite Analysis'}


print("✓ Map IDs generated for all layers")
# Display tile URLs
for layer_name, map_id_dict in generate_map_id({layer_name: vis_param}, {layer_name: image_mosaick})['all_mapid'].items():
    tile_url = map_id_dict['tile_fetcher'].url_format
    print(f"\n{layer_name.upper()}:")
    print(f"  Tile URL: {tile_url}")



✓ Map IDs generated for all layers
Generating GEE Map IDs...

SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR:
  Tile URL: https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/358692236f5e3bf0ac43865a69eb99d7-7ab4b8693eb6f5b1f717b2a814ad1f49/tiles/{z}/{x}/{y}


In [17]:
## META data layers
layers_data = {
    'project_name': project_name,
}

# # Option 1: Simple tile URLs only
# map_layers = {
#     layer_name: map_ids[layer_name]['tile_fetcher'].url_format}

# Option 2 with more detail combination for generating wmts
map_layers = generate_map_id({layer_name: vis_param}, {layer_name: image_mosaick})['map_layers']
map_layers



Generating GEE Map IDs...


{'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/fd68358061bc8b352a9541dc4b084f29-3b6f207a41449a80f2c2588129235134/tiles/{z}/{x}/{y}',
  'name': 'Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor',
  'description': 'SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR visualization from GEE analysis',
  'vis_params': {'bands': ['swir2', 'nir', 'red'],
   'min': 0,
   'max': 0.6,
   'gamma': 1.5}}}

In [18]:
# import sys
# sys.path.append('/usr/src/app/fastapi-gee-service')
from gee_integration import process_gee_to_mapstore

# Import the AOI fix function properly
from gee_lib.osi.utils import process_aoi_geometry

def list_layers_to_wmts(map_layers, AOI):
    """
    Convert map layers to WMTS format
    
    Args:
        map_layers: Dictionary of map layers

    """
    
    # Fix 1: Use correct map layers
    print(f"\n🔍 Available map layers:")
    print(f"   map_layers: {len(map_layers)}")

    # Fix 2: Fix AOI coordinates using the existing function
    print(f"\n🔧 Fixing AOI coordinates...")
    AOI_geom = AOI.geometry()
    aoi_info = process_aoi_geometry(AOI_geom)

    print(f"✅ Fixed AOI Info:")
    print(f"   Bbox: {aoi_info['bbox']}")
    print(f"   Center: {aoi_info['center']}")

    print("🚀 Processing GEE Analysis to MapStore...")
    print(f"   Project: {layers_data['project_name']}")
    print(f"   Layers: {len(map_layers)}")
    print(f"   AOI: {aoi_info['bbox']}")

    # Process the complete integration
    result = process_gee_to_mapstore(
        map_layers=map_layers,
        project_name=layers_data['project_name'],
        aoi_info=aoi_info,
        clear_cache_first=True,
        fastapi_url="http://fastapi:8000"  # Internal Docker network , this should be in localhost, but we can override as well
    )
    return result

result = list_layers_to_wmts(map_layers, AOI) # EQUIVALENT TO MAP.ADDLAYER(RASTER)
result



🔍 Available map layers:
   map_layers: 1

🔧 Fixing AOI coordinates...
✅ Centroid calculated successfully with error margin 1


INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Processing GEE analysis: belaban_rayak
INFO:gee_integration:🧹 Clearing duplicate projects before processing new analysis...
INFO:cache_manager:No existing catalog entries to check for duplicates
INFO:gee_integration:✅ Duplicate clearing successful: 0 duplicates cleared, 0 unique projects kept
INFO:gee_integration:✅ Cache cleared: 0 duplicate entries, kept 0 unique projects
INFO:gee_integration:Using complex layer info for 'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Registering with FastAPI: belaban_rayak_20251027_122600
INFO:gee_integration:✅ FastAPI registration successful: MapStore catalog updated successfully
INFO:gee_integration:Creating FastAPI proxy URLs for project: belaban_rayak_202

Calculated bbox from coordinates: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ AOI processed successfully:
   - Center: [110.46306597258281, -1.803818629688135]
   - Area: Unknown
   - BBox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ Fixed AOI Info:
   Bbox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
   Center: [110.46306597258281, -1.803818629688135]
🚀 Processing GEE Analysis to MapStore...
   Project: belaban_rayak
   Layers: 1
   AOI: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}


INFO:gee_utils:WMTS capabilities refreshed successfully
INFO:gee_utils:Updating MapStore WMTS service...
INFO:gee_utils:Successfully updated MapStore WMTS service: gee_analysis_wmts
INFO:gee_utils:Getting current WMTS layers...
INFO:gee_utils:Comprehensive WMTS refresh completed successfully - Found 1 layers
INFO:gee_integration:✅ MapStore WMTS configuration updated
INFO:gee_integration:   New layers found: 1


{'status': 'success',
 'project_id': 'belaban_rayak_20251027_122600',
 'project_name': 'belaban_rayak',
 'fastapi_registration': {'status': 'success',
  'message': 'MapStore catalog updated successfully',
  'layers_count': 1},
 'proxy_urls_creation': {'status': 'success',
  'message': 'Created 1 proxy URLs',
  'proxy_urls': {'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'proxy_url': 'http://fastapi:8000/tiles/belaban_rayak_20251027_122600/Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/fd68358061bc8b352a9541dc4b084f29-3b6f207a41449a80f2c2588129235134/tiles/{z}/{x}/{y}',
    'layer_name': 'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor',
    'description': 'SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR visualization from GEE analysis'}},
  'layers_count': 1},
 'wmts_configuration': {'status': 'success',
  'message': 'WMTS configuration updated successfully',
  'service_id'

In [19]:
## EXAMPLE TO PUSH GEE TO GEOJSON FAST API and get WFS RESULT, (EQUIVALENT MAP.ADDLAYER (VECTOR))
import requests
import json

# Example GeoJSON data
geojson_data = AOI.getInfo() # important this ee.FeatureCollection should convert to geojson first
name_aoi = 'gee_aoi_dynamic'

link_fastapi = 'http://fastapi:8000/fc/'+name_aoi #only from container, not from localhost

# Push to API
response = requests.post(
    link_fastapi, # running in the notebook for now
    json=geojson_data
)

print('link_localhost: ', 'http://localhost:8001/fc/'+name_aoi)



link_localhost:  http://localhost:8001/fc/gee_aoi_dynamic


In [20]:
# !cat /usr/src/app/mapstore/config/localConfig.json

In [21]:
# # Test the new TMS management system
# from gee_integration import add_tms_layer_to_mapstore, list_gee_tms_layers, clear_all_gee_tms_layers
# print("Adding TMS layer with Direct GEE TILES...")
# # Get the first layer as an example
# first_layer_name = list(map_layers.keys())[0]
# first_layer_info = map_layers[first_layer_name]

# # Add a TMS layer
# tms_result = add_tms_layer_to_mapstore(
#     layer_name=first_layer_name,
#     layer_url=first_layer_info.get('tile_url', ''),
#     use_fastapi_proxy=False, # change this to True if you want to use the fastapi proxy
#     fastapi_pub_url="http://localhost:8001"
# )

# print(f"\n📊 TMS Proxy Result:")
# print(f"   Status: {tms_result['status']}")
# if tms_result['status'] == 'success':
#     print(f"   Service ID: {tms_result['service_id']}")
#     print(f"   Layer Title: {tms_result['layer_title']}")
#     print(f"   TMS URL: {tms_result['tms_url']}")
#     print(f"   Uses FastAPI Proxy: {tms_result['use_fastapi_proxy']}")


In [22]:
# list_gee_tms_layers()
# clear_all_gee_tms_layers()

In [23]:
# path_cog =  os.getenv('GCS_PATH')
## cog gcs to ee image
# import geemap
# drone_cog = geemap.load_GeoTIFF(path_cog)
# type(drone_cog)
# !cat /root/.config/gcloud/application_default_credentials.json
# map_layers = generate_map_id({layer_name: vis_param, 'drone_cog': {}}, {layer_name: image_mosaick, 'drone_cog': drone_cog})['map_layers']
# map_layers
# re add
# result = list_layers_to_wmts(map_layers, AOI)
# result
# Add a TMS layer
# tms_result = add_tms_layer_to_mapstore(
#     layer_name=drone_name,
#     layer_url=first_layer_info.get('tile_url', ''),
#     use_fastapi_proxy=False, # change this to True if you want to use the fastapi proxy
#     fastapi_pub_url="http://localhost:8001"
# )

# drone_name = list(map_layers.keys())[1]
# drone_layer = map_layers[drone_name]
# drone_layer


In [24]:
import geopandas as gpd

### validate the input is correct
##AOI
AOI_shp_path = config['AOI_path']
AOI_shp = gpd.GeoDataFrame.from_file(AOI_shp_path)
AOI_shp.head()

# make sure the related id is exist in the AOI_shp
id_name = config['OID_field_name']
if id_name not in AOI_shp.columns:
    raise ValueError(f"The field {id_name} does not exist in the AOI shapefile.")

# check if the AOI_shp is in the correct crs
if AOI_shp.crs != config['crs_input']:
    AOI_shp = AOI_shp.to_crs(config['crs_input'])



In [25]:
## validate the training points (this notebook assume the training points is already exist in the project)
input_training_points_path = config['input_training']
training_points = gpd.GeoDataFrame.from_file(input_training_points_path)

# check if the training points is in the correct crs
if training_points.crs != config['crs_input']:
    training_points = training_points.to_crs(config['crs_input'])

print('before_validation: ',training_points.shape)
# Function to check if a value is an integer
def is_integer(value):
    return isinstance(value, int)

# Filter out non-integer values in the 'code_lu' column
training_points['code_lu'] = training_points['code_lu'].apply(lambda x: x if is_integer(x) else None)

print('after validation: ',training_points.shape)
training_points.head()


before_validation:  (200, 4)
after validation:  (200, 4)


,id,cd,code_lu,geometry
0,0,3,3,POINT (110.48234 -1.79907)
1,1,4,4,POINT (110.43839 -1.81624)
2,2,1,1,POINT (110.47441 -1.80383)
3,3,1,1,POINT (110.46339 -1.81597)
4,4,1,1,POINT (110.45023 -1.78926)


In [26]:
## we can add the training point to the map (WFS) ;)
training_points_ee = geemap.geopandas_to_ee(training_points)

In [27]:
# Example GeoJSON data, adding again above (points, not only AOI polygon)
geojson_data = training_points_ee.getInfo() # important this ee.FeatureCollection should convert to geojson first
name_aoi = 'training_points_dynamic'

link_fastapi = 'http://fastapi:8000/fc/'+name_aoi #only from container, not from localhost

# Push to API
response = requests.post(
    link_fastapi, # running in the notebook for now
    json=geojson_data
)

In [28]:
pallette_class_segment = {
            '1': '#83ff5a',  # forest_trees (1)
            '2': '#ffe3b3',  # shrubland (2)
            '3': '#ffff33',  # grassland (3)
            '4': '#f89696',  # openland (4)
            '5': '#1900ff',  # waterbody_wet_area (5)
            '6': '#e6e6fa',  # plantation (6)
            '7': '#FFFFFF',   # gray_infrastructure (7)
            '8': '#4B0082',  # oil_palm (8) - Dark Purple
            '9': '#8B4513',  # cropland (9) - Brown
            '10': '#87CEEB',  # waterbody (10) - Light Blue
            '11': '#2F4F4F',  # wetlands (11) - Dark Teal
            '12': '#ADFF2F',  # forest_trees_regrowth (12) - Light Green
            '13': '#8B0000',  # historical_treeloss_10years (13) - Dark Red
            '14': '#DAA520'   # paddy_irrigated (14) - Golden Yellow

            }

training_points['code_lu'].unique()

array([3, 4, 1, 2, 9])

In [29]:
# [f.name for f in map.listLayers()]
# [f.name for f in arc_ops.list_layers]

In [30]:
classImageSpectral = SpectralAnalysis(image_mosaick,config)
class_FCD_run = FCDCalc(config).fcd_calc()
FCD1_1 = class_FCD_run['FCD1_1']
FCD2_1 = class_FCD_run['FCD2_1']

# FCD1_1_arcgis_layer = arc_ops.adding_ee_to_arcgisPro(FCD1_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']},
#                                                    f'FCD1_1_{project_name}')

fcd_visparams = {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}
fcd1_1_layer_name = f'FCD1_1_{project_name}'
fcd2_1_layer_name = f'FCD2_1_{project_name}'

# FCD2_1_arcgis_layer = arc_ops.adding_ee_to_arcgisPro(FCD2_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']},
#                                                    f'FCD2_1_{project_name}')

map_layers = generate_map_id({layer_name: vis_param, fcd1_1_layer_name: fcd_visparams, fcd2_1_layer_name: fcd_visparams}, 
                             {layer_name: image_mosaick, fcd1_1_layer_name: FCD1_1, fcd2_1_layer_name: FCD2_1})['map_layers']

print('finish processing PCA please continue')

selecting Sentinel images
selecting Sentinel images
processing AVI
processing BSI
processing SI
Normalizing to 100 AVI
Normalizing to 100 AVI
Normalizing to 100 BSI
Normalizing to 100 SI
Combining AVI AND BSI
no thermal band, choosing Sentinel images
Processing means center of AVI_BSI please wait
Now we proceed to the PCA of Vegetation density
Success get the PCA normalized of VD => SVI
Now calculating the FCD from SVI and SSI - selecting band svi1 svi2 ssi1 and ssi2
finish processing PCA, the result: FCD1_1 and FCD2_1 please continue
Generating GEE Map IDs...
finish processing PCA please continue


In [31]:
map_layers

{'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/45d939782df899a2c705e1d0bebb6a15-54ebca32acb03c1f55a5770bf8d5d1f4/tiles/{z}/{x}/{y}',
  'name': 'Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor',
  'description': 'SENTINEL MOSAICKED - 2024-1-1-2024-5-31 VEGCOLOR visualization from GEE analysis',
  'vis_params': {'bands': ['swir2', 'nir', 'red'],
   'min': 0,
   'max': 0.6,
   'gamma': 1.5}},
 'FCD1_1_belaban_rayak': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/ecc451a5edb2eebbf38b16b8cef5ecf8-8c8e8e5a26a3675f7452ad15cf9d5105/tiles/{z}/{x}/{y}',
  'name': 'Fcd1 1 Belaban Rayak',
  'description': 'FCD1_1_BELABAN_RAYAK visualization from GEE analysis',
  'vis_params': {'min': 0,
   'max': 80,
   'palette': ['ff4c16', 'ffd96c', '39a71d']}},
 'FCD2_1_belaban_rayak': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/11cf26bbec66dc062ed0ba

In [32]:
result = list_layers_to_wmts(map_layers, AOI) # EQUIVALENT TO MAP.ADDLAYER(RASTER)


🔍 Available map layers:
   map_layers: 3

🔧 Fixing AOI coordinates...
✅ Centroid calculated successfully with error margin 1


INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Processing GEE analysis: belaban_rayak
INFO:gee_integration:🧹 Clearing duplicate projects before processing new analysis...
INFO:cache_manager:No existing catalog entries to check for duplicates
INFO:gee_integration:✅ Duplicate clearing successful: 0 duplicates cleared, 0 unique projects kept
INFO:gee_integration:✅ Cache cleared: 0 duplicate entries, kept 0 unique projects
INFO:gee_integration:Using complex layer info for 'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'FCD1_1_belaban_rayak': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'FCD2_1_belaban_rayak': ['tile_url', 'name', 'description', 'vis_params']
IN

Calculated bbox from coordinates: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ AOI processed successfully:
   - Center: [110.46306597258281, -1.803818629688135]
   - Area: Unknown
   - BBox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ Fixed AOI Info:
   Bbox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
   Center: [110.46306597258281, -1.803818629688135]
🚀 Processing GEE Analysis to MapStore...
   Project: belaban_rayak
   Layers: 3
   AOI: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}


In [33]:
## starting to look at the historical data (Hansen - global foresty watch)
hansen_class = HansenHistorical(config)
run_hansen = hansen_class.initiate_tcl()
LastImageLandsat, treeLossYear, minLoss, ForestArea2000Hansen, gfc =  \
                                 run_hansen['LastImageLandsat'], \
                                 run_hansen['treeLossYear'], \
                                 run_hansen['minLoss'], \
                                 run_hansen['ForestArea2000Hansen'], \
                                 run_hansen['gfc']

In [34]:
## if we want to add the hansen layer to the map
tree_loss_year = treeLossYear.randomVisualizer()

# arc_ops.adding_ee_to_arcgisPro(treeLossYear.randomVisualizer(), {},
#                                                    f'treeLossYear')

treeLoss = treeLossYear.gte(0).And(treeLossYear.lte(23)).selfMask()
# arc_ops.adding_ee_to_arcgisPro(treeLoss.randomVisualizer(), {},
#                                                    f'treeLoss')

#Canopy cover percentage (e.g. 30%), for Indonesia
cc = ee.Number(30)

#Minimum forest area in pixels (e.g. 3 pixels, ~ 0.27 ha in this example).
pixels = ee.Number(3)

#Minimum mapping area for tree loss (usually same as the minimum forest area).
lossPixels = pixels

canopyCover = gfc.select(['treecover2000'])
canopyCoverThreshold = canopyCover.gte(cc).selfMask()

#Use connectedPixelCount() to get contiguous area.
contArea = canopyCoverThreshold.connectedPixelCount()
#Apply the minimum area requirement.
minArea = contArea.gte(pixels).selfMask()

treecoverLoss = minArea.And(treeLoss).rename(f'lossfrom_0-23').selfMask()
        
#Create connectedPixelCount() to get contiguous area.
contLoss = treecoverLoss.connectedPixelCount()
#Apply the minimum area requirement, and get the TCL data ---> minLoss - ACTUAL TCL AREA from Hansen since the year_start_loss
minLoss = contLoss.gte(lossPixels).selfMask()


# arc_ops.adding_ee_to_arcgisPro(minLoss.randomVisualizer(), {},
#                                                    f'minLoss')

In [35]:
### TRY the new Approach, wmts manager, make the simpler line for adding layer based on project
from wmts_manager import WMTSManager

wmts = WMTSManager(project_name=project_name, aoi=AOI.geometry()) # equal to the defining map properties 
wmts.addLayer(image_mosaick, vis_param, layer_name)
wmts.addLayer(FCD1_1, fcd_visparams, fcd1_1_layer_name)
wmts.addLayer(FCD2_1, fcd_visparams, fcd2_1_layer_name)
wmts.addLayer(treeLossYear.randomVisualizer(), {}, f'treeLossYear')
wmts.addLayer(treeLoss.randomVisualizer(), {}, f'treeLoss')
wmts.addLayer(minLoss.randomVisualizer(), {}, f'minLoss')
result = wmts.publish() # execution to the WMTS server

INFO:wmts_manager:WMTSManager initialized for project: belaban_rayak
INFO:wmts_manager:Added layer: Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor
INFO:wmts_manager:Added layer: FCD1_1_belaban_rayak
INFO:wmts_manager:Added layer: FCD2_1_belaban_rayak
INFO:wmts_manager:Added layer: treeLossYear
INFO:wmts_manager:Added layer: treeLoss
INFO:wmts_manager:Added layer: minLoss
INFO:wmts_manager:Generating map IDs for 6 layers...


Generating GEE Map IDs...
✅ Centroid calculated successfully with error margin 1


INFO:wmts_manager:AOI processed: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
INFO:wmts_manager:Publishing 6 layers to WMTS...
INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Processing GEE analysis: belaban_rayak
INFO:gee_integration:🧹 Clearing duplicate projects before processing new analysis...
INFO:cache_manager:No existing catalog entries to check for duplicates
INFO:gee_integration:✅ Duplicate clearing successful: 0 duplicates cleared, 0 unique projects kept
INFO:gee_integration:✅ Cache cleared: 0 duplicate entries, kept 0 unique projects
INFO:gee_integration:Using complex layer info for 'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'FC

Calculated bbox from coordinates: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ AOI processed successfully:
   - Center: [110.46306597258281, -1.803818629688135]
   - Area: Unknown
   - BBox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}


INFO:gee_integration:   New layers found: 6
INFO:wmts_manager:WMTS publishing completed: success


In [36]:
from wfs_manager import WFSManager

wfs = WFSManager(fastapi_url="http://fastapi:8000", wfs_base_url="http://localhost:8001")
wfs.addLayer(training_points_ee, "Training Points Dynamic")
wfs.addLayer(AOI, "AOI Boundary")
result = wfs.publish()

INFO:wfs_manager:WFSManager initialized:
INFO:wfs_manager:  FastAPI URL: http://fastapi:8000
INFO:wfs_manager:  WFS Base URL: http://localhost:8001
INFO:wfs_manager:Added vector layer: Training Points Dynamic -> training_points_dynamic
INFO:wfs_manager:Added vector layer: AOI Boundary -> aoi_boundary
INFO:wfs_manager:Publishing 2 vector layers to WFS...
INFO:wfs_manager:✅ Published layer 'training_points_dynamic': FeatureCollection 'training_points_dynamic' created successfully
INFO:wfs_manager:✅ Published layer 'aoi_boundary': FeatureCollection 'aoi_boundary' created successfully
INFO:wfs_manager:WFS publishing completed: success
INFO:wfs_manager:  Successful: 2
INFO:wfs_manager:  Failed: 0


In [37]:
# NEW (Clean and Flexible):
from osi.utils.main import generate_map_id_list, create_layer_info

# Define your layers in a clean list
layers = [
    create_layer_info(layer_name, image_mosaick, vis_param, "Sentinel Mosaicked Image"),
    create_layer_info(fcd1_1_layer_name, FCD1_1, fcd_visparams, "Forest Cover Density 1"),
    create_layer_info(fcd2_1_layer_name, FCD2_1, fcd_visparams, "Forest Cover Density 2")
]

# Generate Map IDs
result = generate_map_id_list(layers)
map_layers = result['map_layers']
map_layers

Generating GEE Map IDs...
✅ Generated Map ID for layer: Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor
✅ Generated Map ID for layer: FCD1_1_belaban_rayak
✅ Generated Map ID for layer: FCD2_1_belaban_rayak
✅ Generated Map IDs for 3 layers


{'Sentinel mosaicked - 2024-1-1-2024-5-31 VegColor': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/9bc9c23673f70c6addec77db81a23939-71d137d9ab238aa5adcfb565482aecce/tiles/{z}/{x}/{y}',
  'name': 'Sentinel Mosaicked - 2024-1-1-2024-5-31 Vegcolor',
  'description': 'Sentinel Mosaicked Image',
  'vis_params': {'bands': ['swir2', 'nir', 'red'],
   'min': 0,
   'max': 0.6,
   'gamma': 1.5}},
 'FCD1_1_belaban_rayak': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/3b138198c9a16462edfd0c0a34cf56fa-65f2223873cf0f537fcf0425f7dc9594/tiles/{z}/{x}/{y}',
  'name': 'Fcd1 1 Belaban Rayak',
  'description': 'Forest Cover Density 1',
  'vis_params': {'min': 0,
   'max': 80,
   'palette': ['ff4c16', 'ffd96c', '39a71d']}},
 'FCD2_1_belaban_rayak': {'tile_url': 'https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/70e804114cba2ac7504f552e539b7756-3f413b167b9d687daa79ab1dd73b032e/tiles/{z}/{x}/{y}',
  'name': 'Fcd2 1 Bela